# ***Imports***

In [ ]:
import os
import getpass
import json
import re

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

from teradataml import *
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn2pmml import sklearn2pmml, PMMLPipeline
from lightgbm import LGBMClassifier
from tdnpathviz.visualizations import plot_first_main_paths

# Configure BYOM and VAL install locations (crucial for BYOM to work)
configure.byom_install_location = 'mldb'
configure.val_install_location = 'val'

# ***Connect to Teradata Vantage***

In [ ]:
host = '172.16.0.12'
username = getpass.getpass("Username: ")
password = getpass.getpass("Password: ")

# Create connection context
eng = create_context(host=host, username=username, password=password)
print("Connected to Vantage:", eng)

# ***Data exploration***

### Load data from Vantage table

In [ ]:
tdf = DataFrame(in_schema('Source_data_db', 'jcr_fake_events'))

In [ ]:
tdf.head()

In [ ]:
tdf.head().show_query()

### Check data types and basic statistics

In [ ]:
print("Dataset shape:", tdf.shape)
print("\nColumn information:")
tdf.info()
print("\nBasic statistics:")
tdf.describe()

### Visualize data

In [ ]:
# 1. Group by product and count transactions
transactions_by_product = tdf.groupby('product_id').count().get(['product_id', 'count_customer_id'])

# 2. Sort by count in descending order
transactions_by_product = transactions_by_product.sort('count_customer_id', ascending=False)
transactions_by_product = transactions_by_product.assign(
    product_int = case([
        (transactions_by_product.product_id == "Almonds", 0),
        (transactions_by_product.product_id == "Blanket", 1),
        (transactions_by_product.product_id == "Blender", 2),
        (transactions_by_product.product_id == "Charger", 3),
        (transactions_by_product.product_id == "Chocolate", 4),
        (transactions_by_product.product_id == "Cleanser", 5),
        (transactions_by_product.product_id == "Coffee", 6),
        (transactions_by_product.product_id == "Cookies", 7),
        (transactions_by_product.product_id == "Cookware", 8),
        (transactions_by_product.product_id == "Cutlery", 9),
        (transactions_by_product.product_id == "Diffuser", 10),
        (transactions_by_product.product_id == "Granola", 11),
        (transactions_by_product.product_id == "Hairdryer", 12),
        (transactions_by_product.product_id == "Headset", 13),
        (transactions_by_product.product_id == "Honey", 14),
        (transactions_by_product.product_id == "Humidifier", 15),
        (transactions_by_product.product_id == "Kettle", 16),
        (transactions_by_product.product_id == "Keyboard", 17),
        (transactions_by_product.product_id == "Lotion", 18),
        (transactions_by_product.product_id == "Moisturizer", 19),
        (transactions_by_product.product_id == "Mouse", 20),
        (transactions_by_product.product_id == "Oliveoil", 21),
        (transactions_by_product.product_id == "Organizer", 22),
        (transactions_by_product.product_id == "Perfume", 23),
        (transactions_by_product.product_id == "Pillow", 24),
        (transactions_by_product.product_id == "Popcorn", 25),
        (transactions_by_product.product_id == "Router", 26),
        (transactions_by_product.product_id == "Serum", 27),
        (transactions_by_product.product_id == "Shampoo", 28),
        (transactions_by_product.product_id == "Skincare", 29),
        (transactions_by_product.product_id == "Speaker", 30),
        (transactions_by_product.product_id == "Sunscreen", 31),
        (transactions_by_product.product_id == "Tea", 32),
        (transactions_by_product.product_id == "Toaster", 33),
        (transactions_by_product.product_id == "Toothbrush", 34),
        (transactions_by_product.product_id == "Truffles", 35),
        (transactions_by_product.product_id == "Webcam", 36),
    ])
)

In [ ]:
# transactions_by_product = tdf.groupby('product_id').count().get(['product_id', 'count_customer_id'])
# transactions_by_product = transactions_by_product.sort('count_customer_id', ascending=False)
# # Example alternative: map product categories to their index dynamically
# product_categories = [
#     "Almonds", "Blanket", "Blender", "Charger", "Chocolate", "Cleanser", "Coffee", 
#     "Cookies", "Cookware", "Cutlery", "Diffuser", "Granola", "Hairdryer", "Headset", 
#     "Honey", "Humidifier", "Kettle", "Keyboard", "Lotion", "Moisturizer", "Mouse",
#     "Oliveoil", "Organizer", "Perfume", "Pillow", "Popcorn", "Router", "Serum", 
#     "Shampoo", "Skincare", "Speaker", "Sunscreen", "Tea", "Toaster", "Toothbrush", 
#     "Truffles", "Webcam"
# ]
# mapping_expr = case([
#     (transactions_by_product.product_id == prod, idx) 
#     for idx, prod in enumerate(product_categories)
# ])
# transactions_by_product = transactions_by_product.assign(product_int = mapping_expr)

In [ ]:
# 1. Product Purchase Frequency - Bar Plot (By Product index)
transactions_by_product.plot(
    x=transactions_by_product.product_int,
    y=transactions_by_product.count_customer_id,
    kind='bar',
    title="Number of Transactions per Product",
    ylabel='Count of Transactions',
    xlabel='Product (indexed)'
)

In [ ]:
# Convert to Pandas DataFrame
pdf = transactions_by_product.to_pandas(all_rows=True)

# Use Teradata .plot() but with custom parameters for better appearance
pdf.plot(
    x='product_id',
    y='count_customer_id',
    kind='bar',
    color='skyblue',   # Bar color
    legend=False,
    figsize=(10, 5),   # Similar to plt.figure(figsize=(10,5))
    title="Product Purchase Frequency",
    xlabel='Product ID',
    ylabel='Number of Purchases'
).tick_params(axis='x', rotation=45)

In [ ]:
# Create aggregated data for product frequency
product_freq = df1.groupby(['product_id']).count()
product_freq = product_freq.assign(purchase_count=product_freq.count_customer_id)
product_freq = product_freq.select(['product_id', 'purchase_count'])
product_freq = product_freq.sort(['purchase_count'], ascending=False)

# Filter top 10 products for better visualization
df2 = product_freq.head(10)
df2.plot(x=df2.product_id, 
        y=df2.purchase_count, 
        kind="bar",
        title="Top 10 Products by Purchase Frequency",
        color="skyblue",
        xlabel="Product ID",
        ylabel="Purchase Count",
        grid_linestyle="-",
        grid_linewidth=0.5 
        )

In [ ]:
#2. Line PLot
pdf = tdf.to_pandas(all_rows=True)

# Convert timestamp to datetime
pdf['ts'] = pd.to_datetime(pdf['ts'])

# Aggregate purchases per day
daily_counts = pdf.groupby(pdf['ts'].dt.date).size().reset_index(name='purchase_count')

# Plot using pandas built-in plotting (similar to df1.plot)
plot = daily_counts.plot(
    x='ts', 
    y='purchase_count', 
    kind='line',
    title="Purchases Over Time",
    legend=False,
    xlabel="Date",
    ylabel="Number of Purchases",
    figsize=(12, 6)
    
).tick_params(axis='x', rotation=45)


# ***Analyze the sequence of purchase of products by a customer***

# Copy to Vantage for nPath

In [ ]:
copy_to_sql(tdf[['customer_id', 'product_id', 'price' ,'ts']], table_name='npath_purchase_data', if_exists='replace')

# Run NPath to detect sequential purchases

In [ ]:
npath_result = NPath(
    data1=DataFrame("npath_purchase_data"),
    data1_partition_column=["customer_id"],
    data1_order_column=["customer_id","ts"],
    mode="NONOVERLAPPING",
    symbols=["TRUE AS A", "TRUE AS B"],
    pattern="(A){4}.B",
    result=[
        "ACCUMULATE(product_id OF A) AS current_products",
        "FIRST(customer_id OF A) AS customer_id",
    ]
)

purchase_pairs_df = npath_result.result
purchase_pairs_df

In [ ]:
plot_first_main_paths(purchase_pairs_df, path_column = 'current_products', id_column = 'customer_id',width=1000)

# ***Generate target variable***

## Create a feature table that includes RFM-based features (recency, frequency, and monetary values)

In [ ]:
copy_to_sql(tdf[['customer_id', 'product_id', 'price' ,'ts']], table_name='customer_purchase_data', if_exists='replace')

In [ ]:
# 1. Last Purchase Date per Customer
query_last_purchase = '''
CREATE VOLATILE TABLE last_purchase AS
(
    SELECT
        customer_id,
        MAX(ts) AS last_purchase_ts
    FROM customer_purchase_data
    GROUP BY customer_id
) WITH DATA ON COMMIT PRESERVE ROWS;
'''
execute_sql(query_last_purchase)

In [ ]:
quey_purchase_events= '''
CREATE VOLATILE TABLE purchase_events AS
(
    SELECT
        customer_id,
        ts,
        product_id AS last_product,
        LEAD(product_id) OVER (PARTITION BY customer_id ORDER BY ts) AS next_product
    FROM customer_purchase_data
) WITH DATA ON COMMIT PRESERVE ROWS;
'''
execute_sql(quey_purchase_events)

In [ ]:
# 2. Global Max Timestamp
query_max_ts = '''
CREATE VOLATILE TABLE max_ts_table AS
(
    SELECT MAX(ts) AS max_ts
    FROM customer_purchase_data
) WITH DATA ON COMMIT PRESERVE ROWS;
'''
execute_sql(query_max_ts)


In [ ]:
# 3. Recency
query_recency = '''
CREATE VOLATILE TABLE recency_per_purchase AS
(
    SELECT
        pe.customer_id,
        pe.ts,
        CASE 
            WHEN LAG(pe.ts) OVER (PARTITION BY pe.customer_id ORDER BY pe.ts) IS NULL THEN NULL
            ELSE CAST(pe.ts AS DATE) - CAST(LAG(pe.ts) OVER (PARTITION BY pe.customer_id ORDER BY pe.ts) AS DATE)
        END AS recency_days
    FROM purchase_events pe
) WITH DATA ON COMMIT PRESERVE ROWS;
'''
execute_sql(query_recency)


In [ ]:
# 4. Frequency (6 months)
query_frequency = '''
CREATE VOLATILE TABLE frequency_per_purchase AS
(
    SELECT
        pe.customer_id,
        pe.ts,
        COUNT(*) AS frequency_6m
    FROM purchase_events pe
    JOIN customer_purchase_data p
      ON pe.customer_id = p.customer_id
     AND CAST(p.ts AS DATE) BETWEEN ADD_MONTHS(CAST(pe.ts AS DATE), -6) AND CAST(pe.ts AS DATE)
    GROUP BY pe.customer_id, pe.ts
) WITH DATA ON COMMIT PRESERVE ROWS;
'''
execute_sql(query_frequency)


In [ ]:
# 5. Monetary Value (6 months)
query_monetary = '''
CREATE VOLATILE TABLE monetary_per_purchase AS
(
    SELECT
        pe.customer_id,
        pe.ts,
        AVG(p.price) AS avg_price_6m,
        SUM(p.price) AS total_price_6m
    FROM purchase_events pe
    JOIN customer_purchase_data p
      ON pe.customer_id = p.customer_id
     AND CAST(p.ts AS DATE) BETWEEN ADD_MONTHS(CAST(pe.ts AS DATE), -6) AND CAST(pe.ts AS DATE)
    GROUP BY pe.customer_id, pe.ts
) WITH DATA ON COMMIT PRESERVE ROWS;

'''
execute_sql(query_monetary)


In [ ]:
# 6. Average Days Between Purchases
query_avg_days_between = '''
CREATE VOLATILE TABLE avg_days_between_per_purchase AS
(
    SELECT
        pe.customer_id,
        pe.ts,
        AVG(CAST(p2.ts AS DATE) - CAST(p1.ts AS DATE)) AS avg_days_between_purchase
    FROM purchase_events pe
    JOIN customer_purchase_data p1
      ON pe.customer_id = p1.customer_id AND p1.ts <= pe.ts
    JOIN customer_purchase_data p2
      ON pe.customer_id = p2.customer_id AND p2.ts <= pe.ts
         AND p2.ts > p1.ts
    GROUP BY pe.customer_id, pe.ts
) WITH DATA ON COMMIT PRESERVE ROWS;

'''
execute_sql(query_avg_days_between)


In [ ]:
# 8. Product Diversity
query_product_diversity = '''
CREATE VOLATILE TABLE product_diversity_per_purchase AS
(
    SELECT
        pe.customer_id,
        pe.ts,
        COUNT(DISTINCT p.product_id) AS product_diversity
    FROM purchase_events pe
    JOIN customer_purchase_data p
      ON pe.customer_id = p.customer_id AND p.ts <= pe.ts
    GROUP BY pe.customer_id, pe.ts
) WITH DATA ON COMMIT PRESERVE ROWS;

'''
execute_sql(query_product_diversity)


In [ ]:
# 10. Final Feature Table
query_final_features = '''
CREATE MULTISET TABLE final_features AS
(
    SELECT 
        pe.customer_id,
        pe.ts,
        r.recency_days,
        f.frequency_6m,
        m.avg_price_6m,
        m.total_price_6m,
        adb.avg_days_between_purchase,
        pe.last_product,
        pd.product_diversity,
        pe.next_product
    FROM purchase_events pe
    LEFT JOIN recency_per_purchase r ON pe.customer_id = r.customer_id AND pe.ts = r.ts
    LEFT JOIN frequency_per_purchase f ON pe.customer_id = f.customer_id AND pe.ts = f.ts
    LEFT JOIN monetary_per_purchase m ON pe.customer_id = m.customer_id AND pe.ts = m.ts
    LEFT JOIN avg_days_between_per_purchase adb ON pe.customer_id = adb.customer_id AND pe.ts = adb.ts
    LEFT JOIN product_diversity_per_purchase pd ON pe.customer_id = pd.customer_id AND pe.ts = pd.ts
    WHERE pe.next_product IS NOT NULL  -- Only rows with a known next product
) WITH DATA NO PRIMARY INDEX;
'''
execute_sql(query_final_features)

### Load Final Features Table

In [ ]:
tdf_final_features=DataFrame("final_features")
tdf_final_features.head()

### Check data types and basic statistics

In [ ]:
print("Dataset shape:", tdf_final_features.shape)
print("\nColumn information:")
tdf_final_features.info()
print("\nBasic statistics:")
tdf_final_features.describe()

### Checking for Null Values

In [ ]:
colsum = ColumnSummary(
    data  = tdf_final_features,
    target_columns = [':']
)
colsum.result

### Handle Missing Values with Imputation

In [ ]:
# Step 1: Fit imputer
impute_fit_output = SimpleImputeFit(
    data=tdf_final_features,
    literals_columns="avg_days_between_purchase",
    literals="0"
)

# Step 2: Apply transformation
tdf_final_features_clean = SimpleImputeTransform(
    data=tdf_final_features,
    object=impute_fit_output.output
).result   # This is now a teradataml DataFrame

In [ ]:
colsum = ColumnSummary(
    data  = tdf_final_features_clean,
    target_columns = [':']
)
colsum.result

## Data Preprocessing

### Encode Categorical Features (One-Hot Encoding)

In [ ]:
# Create fit object to encode "last_product" column
hot_fit = OneHotEncodingFit(
    data=tdf_final_features_clean,
    is_input_dense=True,
    target_column=['last_product'],
    category_counts=[37],  # Must be a list
    approach="auto",
    other_column="other"
)

# Print the result DataFrame
hot_fit.result
# Apply the one-hot encoding using the fit object
hot_encoded = OneHotEncodingTransform(
    data=tdf_final_features_clean,
    object=hot_fit,          # Use the fit object
    is_input_dense=True      # Required parameter
)

# Show the transformed dataset
hot_encoded.result


### Encode Target Column (Ordinal Encoding)

In [ ]:
# Create ordinal encoding fit object for next_product
ordinal_fit = OrdinalEncodingFit(
    target_column=['next_product'],   # Column to encode
    data=tdf_final_features_clean,
    default_value=-1                  # For unknown/unseen categories
)

# Show the mapping result
ordinal_fit.result
ordinal_encoded = OrdinalEncodingTransform(
    data=tdf_final_features_clean,
    object=ordinal_fit
)

# View transformed dataset
ordinal_encoded.result


### Scale Numerical Features

In [ ]:
# Apply scaling on numeric columns (example: recency_days)
scale_fit = ScaleFit(
    data=tdf_final_features_clean,
    target_columns=[
        "recency_days", 
        "frequency_6m", 
        "avg_price_6m", 
        "total_price_6m", 
        "avg_days_between_purchase",
        "product_diversity"
    ],
    scale_method="RANGE",   # Scale to [0,1] range
    miss_value="KEEP",      # Keep missing values
    global_scale=False      # Scale each column independently
)

# Show the scaling output (min/max, etc.)
scale_fit.output
scale_transformed = ScaleTransform(
    data=tdf_final_features_clean,
    object=scale_fit
)

# View transformed scaled data
scale_transformed.result


## Feature Combination and Training/Testing Dataset Creation

In [ ]:
# Combine scaling, one-hot, and ordinal encoding
combined_transform = ColumnTransformer(
    input_data=tdf_final_features_clean,
    scale_fit_data=scale_fit.output,           # Use .output here
    onehotencoding_fit_data=hot_fit.result,    # Use .result for OneHotEncodingFit
    ordinalencoding_fit_data=ordinal_fit.result  # Use .result for OrdinalEncodingFit
)

# Final transformed DataFrame
tdf_transformed = combined_transform.result
tdf_transformed

In [ ]:
cols = [c for c in tdf_transformed.columns if c != 'next_product'] + ['next_product']
tdf_transformed = tdf_transformed[cols]

In [ ]:
# 1. Drop columns not needed for modeling
tdf_dataset = tdf_transformed.drop(columns=['customer_id', 'ts', 'last_product', 'last_product_other'])

# 2. Split dataset into train and test
tdf_sample = tdf_dataset.sample(frac=[0.8, 0.2])
df_train = tdf_sample[tdf_sample["sampleid"] == 1].drop("sampleid", axis=1)
df_test  = tdf_sample[tdf_sample["sampleid"] == 2].drop("sampleid", axis=1)

In [ ]:
print("Training Set = " + str(df_train.shape[0]) + ". Testing Set = " + str(df_test.shape[0]))

In [ ]:
copy_to_sql(df_train, table_name = 'final_data_train', if_exists = 'replace')
copy_to_sql(df_test, table_name = 'final_data_test', if_exists = 'replace')

In [ ]:
df_train

In [ ]:
df_test

# ***Generate a predictive model using lightGBM and sklearn***

# Model Training with LightGBM

In [ ]:
final_data_train_df = df_train.to_pandas(all_rows = True)
final_data_train_df.reset_index(inplace = True)

In [ ]:

# Split into features (X_train) and target (y_train)
y_train = final_data_train_df.iloc[:, -1]
X_train = final_data_train_df.iloc[:, :-1]


In [ ]:
# Define LightGBM as a Scikit-learn classifier
lgbm_model = LGBMClassifier(
    objective="multiclass",
    num_class=len(y_train.unique()),
    learning_rate=0.05,
    num_leaves=31,
    n_estimators=100
)

# Create PMML pipeline and train
pipeline = PMMLPipeline([
    ("classifier", lgbm_model)
])
pipeline.fit(X_train, y_train)

# Export to PMML
sklearn2pmml(pipeline, "lgbm_next_product.pmml", with_repr=True)
print("Model trained and saved as lgbm_next_product.pmml")

# ***Import model in Vantage***

In [ ]:
# Load the PMML model into Vantage
try:
    res = save_byom(
        model_id='lgbm_next_product',       # Unique ID for your model
        model_file='lgbm_next_product.pmml', # PMML file from sklearn2pmml
        table_name='lgbm_models'            # Table to store the model
    )

except Exception as e:
    # If the model ID already exists, delete and overwrite
    if str(e.args).find('TDML_2200') >= 1:
        res = delete_byom(model_id='lgbm_next_product', table_name='lgbm_models')
        res = save_byom(model_id='lgbm_next_product', model_file='lgbm_next_product.pmml', table_name='lgbm_models')
        pass
    else:
        raise

In [ ]:
DataFrame('lgbm_models')

# ***Score the model in-database***

In [ ]:
# 1. Retrieve the LightGBM model from Vantage
model_tdf = retrieve_byom("lgbm_next_product", table_name='lgbm_models')

# 2. Run predictions on df_test
result = PMMLPredict(
    modeldata=model_tdf,       
    newdata=df_test,           
    accumulate=['recency_days', 'frequency_6m', 'avg_price_6m', 
                'total_price_6m', 'avg_days_between_purchase', 
                'product_diversity'],  
    overwrite_cached_models='*'
)

In [ ]:
result.result

## Extracting Predicted Class from Model Output

In [ ]:
df = result.result.to_pandas(all_rows=True)

# Find class with max probability
def get_top_class(prob_json):
    probs = json.loads(prob_json)
    return max(probs, key=probs.get)  # e.g., "probability(20)"

# Extract numeric part
def extract_number(prob_str):
    match = re.search(r'\d+', prob_str)
    return int(match.group()) if match else None

df['predicted_class'] = df['json_report'].apply(get_top_class)
df['predicted_class_num'] = df['predicted_class'].apply(extract_number)

print(df[['predicted_class_num', 'json_report']].head())

# ***Cleanup***

In [ ]:
# List of tables to drop
tables_to_drop = [
    "npath_purchase_data",
    "customer_purchase_data",
    "last_purchase",
    "purchase_events",
    "max_ts_table",
    "recency_per_purchase",
    "frequency_per_purchase",
    "monetary_per_purchase",
    "avg_days_between_per_purchase",
    "product_diversity_per_purchase",
    "final_features",
    "final_data_train",
    "final_data_test",
    "lgbm_models"
]

# Drop each table if it exists
for table in tables_to_drop:
    try:
        db_drop_table(table_name=table)
        print(f"Table '{table}' dropped successfully.")
    except:
        print(f"Table '{table}' does not exist or could not be dropped.")

In [ ]:
remove_context()